# Graph Aggregation

## Getting Data

In [1]:
using OpenStreetMapX
using JLD
using CSV
using DataFrames
using StatsBase
using JuMP
using Gurobi
using H3.API

include("../src/data/osm.jl")
include("../src/data/distance_matrix.jl")
include("../src/optimisation/mip/p_mp_model.jl")
include("../src/optimisation/metrics.jl")
include("../src/optimisation/iterative/optimisation.jl")

location_optimization_radius (generic function with 5 methods)

In [2]:
LOCATION = "winnipeg"
DIR_PATH = "../osm_maps/" * LOCATION * "/"
OSM_PATH = DIR_PATH * "map.osm"

isdir(DIR_PATH) ? nothing : mkdir(DIR_PATH)
isfile(OSM_PATH) ? nothing : get_osm_by_bbox(-97.3581, 49.7147, -96.9475, 49.9954, OSM_PATH)

In [3]:
mx = get_map_data(OSM_PATH, use_cache=true, road_levels=Set(1:5), trim_to_connected_graph = true)

┌ Info: Read map data from cache ../osm_maps/winnipeg\map.osm.cache
└ @ OpenStreetMapX C:\Users\Kiryl\.julia\packages\OpenStreetMapX\Yo58L\src\parseMap.jl:93


MapData(OpenStreetMapX.Bounds{LLA}(49.7147, 49.9954, -97.3581, -96.9475), Dict{Int64, ENU}(7799632408 => ENU(1494.130907217481, 4152.141303999004, -1.5273067814109709), 285736360 => ENU(1101.5882285733383, -15065.239259369318, -17.90216221123228), 1488823329 => ENU(7897.341566968542, -2778.649740685668, -5.485410342386103), 741689169 => ENU(-12734.403532880367, -9770.036591763856, -20.17696398425869), 256153327 => ENU(-7823.9314265355515, 4328.561832938522, -6.259356765273878), 631381980 => ENU(-1886.773140262505, 12270.444491945469, -12.091470101295272), 2675696260 => ENU(4681.149026966116, 8111.047167332585, -6.876158216364274), 136147026 => ENU(5277.55875406265, 3364.7416652434094, -3.067432797258334), 1227198431 => ENU(2756.332940129022, 6635.241107579223, -4.048647734779024), 1112293467 => ENU(1331.3070969172638, -797.750121809354, -0.18860118046427488)…), OpenStreetMapX.Way[OpenStreetMapX.Way(4443871, [27286480, 27286484, 34040347, 27286488, 27366907, 27286482, 27366892, 34040311

## Processing Data

In [4]:
TREES_PATH = DIR_PATH * "trees.jld"
METRIC = "time"

if isfile(TREES_PATH)
    node_data = load(TREES_PATH, "node_data")
else
    node_data = create_distance_trees(mx, METRIC)
    save(TREES_PATH, "node_data", node_data)
end

Dict{Int64, Tuple{Vector{Int64}, Vector{Float64}}} with 5752 entries:
  2428190804 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  128583108  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  285736360  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  6598934129 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  741689169  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  2709909467 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  1658443102 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  1115858543 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  278613704  => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  4785952268 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  3131822521 => ([2428190804, 128583108, 285736360, 6598934129, 741689169, 2709…
  1512335051 => ([2428190804, 128583108

In [5]:
DISTANCE_MATRIX_PATH = DIR_PATH * "distance_matrix.jld"
NODE_IDS_PATH = DIR_PATH * "node_ids.jld"


if isfile(DISTANCE_MATRIX_PATH) & isfile(NODE_IDS_PATH)
    all_node_ids = load(NODE_IDS_PATH, "all_node_ids")
    distance_matrix = load(DISTANCE_MATRIX_PATH, "distance_matrix")
else
    all_node_ids = collect(keys(node_data))    
    distance_matrix = build_distance_matrix_graph(all_node_ids, all_node_ids, node_data)
end

5752×5752 Matrix{Int64}:
    0  387  1028   389  1154   588  …   797   787   517  723   830   904
  418    0   713   539   860   517      669   660   130  429   466   777
 1048  770     0  1130   796  1087     1066  1138   682  620   549  1254
  396  549  1133     0  1184   552      770   760   670  754   889   877
 1153  917   764  1181     0  1010      702   791   850  457   608   674
  567  543  1097   561  1011     0  …   553   543   646  581   732   660
  563  549  1103   269  1154   288      695   685   652  724   859   802
  767  674  1071   768   739   568       85   214   750  482   633   285
  672  579   894   673   735   460      170   299   608  305   456   371
  727  415   353   775   592   732      710   782   327  265   291   899
 1135  871   613  1231   424  1096  …   993  1065   783  547   461  1019
  783  471   344   831   583   789      767   839   383  321   335   956
  622  578   854   983  1079  1040     1193  1184   592  827   831  1300
    ⋮                     

## Geo to H3 conversion

In [6]:
H3_RESOLUTION = 9

nodes_ids = [x for x in keys(mx.v)] #finding all node ids
geo_points = [(LLA(mx.nodes[x], mx.bounds).lat, LLA(mx.nodes[x], mx.bounds).lon) for x in nodes_ids] # for each node converting to lat, lng
h3s = [geoToH3(GeoCoord(deg2rad(x[1]), deg2rad(x[2])), H3_RESOLUTION) for x in geo_points] # geo point -> h3
unique_h3s = unique(h3s)
h3_centroids = [(rad2deg(h3ToGeo(x).lat), rad2deg(h3ToGeo(x).lon)) for x in unique_h3s] # centroid of each h3
centroids_nodes = [point_to_nodes(x, mx) for x in h3_centroids] # each centroid -> node id

1402-element Vector{Int64}:
           369959746
           503628052
           285736360
          7217517351
          7539516274
          2709909467
          1439029916
          1115858543
           367112902
           764639249
          3131822521
          1512335077
           128868001
                   ⋮
 1022857356427403792
           467808583
            81960916
          2086667444
           272499055
           128201569
          2783567430
           367039628
           281136905
          1705375652
          2011366697
           339928859

In [7]:
agg_distance_matrix = build_distance_matrix_graph(centroids_nodes, centroids_nodes, node_data)

1402×1402 Matrix{Int64}:
    0   389  1021   373  1146   581  …  1147   469   543   248   283   743
  530     0   704   640   861   627     1137   617   560   750   784   376
 1054   713     0  1120   795  1087     1504  1097  1020  1275  1298   560
  394   540  1124     0  1175   544     1118   270   514   274   654   799
 1164   858   755  1175     0  1014      792  1152   906  1384  1418   613
  574   542  1097   551  1010     0  …   910   351   169   801   857   733
  566   552  1107   263  1157   292     1004   126   299   511   850   782
  774   674  1071   758   737   568      535   734   460  1022  1057   634
  683   552   899   668   739   465      628   644   356   931   966   462
  738   362   354   770   591   737     1154   746   669   959   992   209
 1142   814   613  1222   423  1096  …  1134  1198   988  1362  1385   566
  795   419   349   827   587   794     1210   803   726  1015  1049   266
  514   472   739   859   963   926     1435   915   858   734   765   657


In [8]:
LOCATION = "winnipeg_agg_" * string(H3_RESOLUTION)
DIR_PATH = "../osm_maps/" * LOCATION * "/"
DISTANCE_MATRIX_PATH = DIR_PATH * "distance_matrix.jld"
NODE_IDS_PATH = DIR_PATH * "node_ids.jld"

save(DISTANCE_MATRIX_PATH, "distance_matrix", agg_distance_matrix)
save(NODE_IDS_PATH, "all_node_ids", centroids_nodes)